In [1]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [2]:
df_ratings = pd.read_csv('./ratings.csv', sep=',', usecols=['userId', 'movieId', 'rating'])
df_titles = pd.read_csv("./movies.csv", sep=',', usecols=['movieId', 'title'])

In [3]:
df = pd.merge(df_ratings, df_titles, on='movieId')


In [4]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [5]:
trainset, testset = train_test_split(data, test_size=0.15)

In [6]:
sim_options = {
    'name': 'cosine',
    'user_based': False  # Compute similarities between items
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...


MemoryError: Unable to allocate 12.2 GiB for an array with shape (57299, 57299) and data type int32

In [ ]:
movie_id_to_title = dict(zip(df_titles['movieId'], df_titles['title']))
title_to_movie_id = dict(zip(df_titles['title'], df_titles['movieId']))

In [ ]:
def get_similar_movies(movie_title, num_recommendations=5):
    movie_id = title_to_movie_id[movie_title]
    movie_inner_id = algo.trainset.to_inner_iid(movie_id)
    movie_neighbors = algo.get_neighbors(movie_inner_id, k=num_recommendations)
    similar_movies = [algo.trainset.to_raw_iid(inner_id) for inner_id in movie_neighbors]
    similar_movie_titles = [movie_id_to_title[movie_id] for movie_id in similar_movies]
    return similar_movie_titles

In [ ]:
movie_title = "Toy Story (1995)"  # Replace with the movie title you want to test
recommendations = get_similar_movies(movie_title, num_recommendations=5)
print(f"Top 5 recommendations for the movie '{movie_title}': {recommendations}")